In [ ]:
%matplotlib inline

import cv2
import numpy as np
import scipy

import seaborn as sns

import matplotlib.pyplot as plt

import utils
import stitching_functions

In [ ]:
data_ix = 0
if data_ix == 0:
    nndr_threshold = 0.35
    step_size = 0.001
    residual_stopping_threshold = 0.1
    img_list = [cv2.imread("dataset/data_image_stitching/im1.png"),
                cv2.imread("dataset/data_image_stitching/im2.png")]

elif data_ix == 1:
    nndr_threshold = 0.40
    step_size = 0.001
    residual_stopping_threshold = 10
    img_list = [cv2.imread("dataset/data_image_stitching/im22.jpg"),
                cv2.imread("dataset/data_image_stitching/im23.jpg")]

elif data_ix == 2:
    nndr_threshold = 0.25
    step_size = 0.001
    residual_stopping_threshold = 0.1
    img_list = [cv2.imread("output/stitched_img_1.png"),
                cv2.imread("dataset/data_image_stitching/im24.jpg")]

elif data_ix == 3:
    nndr_threshold = 0.33
    step_size = 0.01
    residual_stopping_threshold = 0.02
    img_list = [cv2.imread("output/stitched_img_2.png"),
                cv2.imread("dataset/data_image_stitching/im25.jpg")]

elif data_ix == 4:
    nndr_threshold = 0.35
    step_size = 0.001
    residual_stopping_threshold = 0.1
    img_list = [cv2.imread("dataset/data_image_stitching/im89.jpg"),
                cv2.imread("dataset/data_image_stitching/im90.jpg")]

In [ ]:
fig = plt.figure(figsize=(8*len(img_list), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_list)):
    plt.subplot(1, len(img_list), i+1)
    plt.imshow(cv2.cvtColor(img_list[i], cv2.COLOR_BGR2RGB))

In [ ]:
keypoints, descriptors, img_keypoints = utils.find_keypoints(img_list)
fig = plt.figure(figsize=(8*len(img_keypoints), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_keypoints)):
    plt.subplot(1, len(img_keypoints), i+1)
    plt.imshow(cv2.cvtColor(img_keypoints[i], cv2.COLOR_BGR2RGB))

In [ ]:
matched_points, matches1to2, eligible_matches = utils.find_matches(descriptors)

filtered_keypoints = []
filtered_keypoints.append([keypoints[0][match[0]] for match in matched_points])
filtered_keypoints.append([keypoints[1][match[1]] for match in matched_points])
img_keypoints = []
img_keypoints.append(cv2.drawKeypoints(img_list[0], filtered_keypoints[0], None))
img_keypoints.append(cv2.drawKeypoints(img_list[1], filtered_keypoints[1], None))

fig = plt.figure(figsize=(8*len(img_keypoints), 8))
fig.patch.set_facecolor('white')
for i in range(len(img_keypoints)):
    plt.subplot(1, len(img_keypoints), i+1)
    plt.imshow(cv2.cvtColor(img_keypoints[i], cv2.COLOR_BGR2RGB))

In [ ]:
matching_img = cv2.drawMatches(img_list[0], filtered_keypoints[0], img_list[1], filtered_keypoints[1], matches1to2, None)
fig = plt.figure(figsize=(16, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(matching_img, cv2.COLOR_BGR2RGB))

In [ ]:
use_package = False

if use_package:
    (homography, mask) = cv2.findHomography(np.float32([kp.pt for kp in filtered_keypoints[1]]), 
                                            np.float32([kp.pt for kp in filtered_keypoints[0]]), 
                                            cv2.RANSAC, ransacReprojThreshold=3.0)

else:
    homography = stitching_functions.find_homography(eligible_matches, keypoints[0], keypoints[1], step_size, residual_stopping_threshold)

t, ht, ymax, ymin, xmax, xmin, height1, width1 = stitching_functions.perspective_transform(img_list, homography)
stitch_r = stitching_functions.warp_perspective(img_list, ymax, ymin, xmax, xmin, ht, homography)
fig = plt.figure(figsize=(12, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(stitch_r, cv2.COLOR_BGR2RGB))

In [ ]:
stitch_l = np.zeros_like(stitch_r)
stitch_l[t[1]:height1 + t[1], t[0]:width1 + t[0]] = img_list[0]
fig = plt.figure(figsize=(12, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(stitch_l, cv2.COLOR_BGR2RGB))

In [ ]:
result = stitching_functions.alpha_blending(stitch_r, stitch_l)
fig = plt.figure(figsize=(12, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(result, cv2.COLOR_BGR2RGB))

In [ ]:
clipped_img = stitching_functions.clip_image(result, t)
fig = plt.figure(figsize=(12, 8))
fig.patch.set_facecolor('white')
plt.imshow(cv2.cvtColor(clipped_img, cv2.COLOR_BGR2RGB))

In [ ]:
cv2.imwrite(f"output/stitched_img_{data_ix}.png", clipped_img)